In [ ]:
import os
os.chdir("/home/comsee/PhD_Antony/.../neuvisys_analysis/")

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import pprint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pdf2image import convert_from_path
from src.spiking_network.network.neuvisys import SpikingNetwork
from src.spiking_network.analysis.network_display import (
    display_network,
    pdf_complex_receptive_fields,
)
from src.spiking_network.gabor2 import fit
import theano
import matplotlib.image as mpimg
from  matplotlib.colors import LinearSegmentedColormap
from fpdf import FPDF

os.chdir("/home/.../neuvisys/neuvisys-analysis/src")
home = "/home/.../"

SMALL_SIZE = 20
MEDIUM_SIZE = 25
BIGGER_SIZE = 30

plt.rc('figure', figsize=(30, 8))
plt.rc('font', size=SMALL_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE,
       labelsize=MEDIUM_SIZE)  # fontsize of the axes title, fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize

In [ ]:
gf = fit.GaborFit()

In [ ]:
networks_path3 = "/.../"

path = networks_path3 + ".../"

spinet = SpikingNetwork(path)

In [ ]:
display_network([spinet])

print("Simple cells weights")
if spinet.conf["nbCameras"] == 2:
    display(convert_from_path(path + "figures/0/weight_sharing_combined.pdf")[0])
else:
    display(convert_from_path(path + "figures/0/weight_sharing_0.pdf")[0])

print("Complex cells weights (average over all simple cells)")
for i in range(spinet.l_shape[1, 2]):
    display(convert_from_path(path + "figures/1/complex_weights_depth_" + str(i) + ".pdf")[0])
    break

In [ ]:
pdf_complex_receptive_fields(spinet,1)

In [ ]:
basis = spinet.generate_weight_mat2()

In [ ]:
cmap=LinearSegmentedColormap.from_list('rg',["r", "k", "lime"], N=256) 

In [ ]:
camera = 0
source_path = spinet.path + "images/0/gabors/"
errors = []
for idex, b in enumerate(basis):
    neuron = spinet.neurons[0][idex]
    path = source_path + str(idex) + ".png"
    _, pp, se = gf.fit(np.expand_dims(b, axis = 2)) 
    pp = fit.combine_params(*pp)
    gs = gf.make_gabor(pp, 10,10)
    gs = np.squeeze(gs)
    
    fig, axes = plt.subplots(2, 1)
    
    axes[0].axis("off")
    axes[1].axis("off")
    axes[0].imshow(mpimg.imread(neuron.weight_images[camera]))
    axes[1].imshow(np.transpose(gs), cmap = cmap)
    plt.savefig(path, bbox_inches="tight")
    plt.close(fig)
    
    errors.append(se[0])
